In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls  # Check if required cuda 9.0 amd64-deb file is downloaded
!dpkg -i cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls /var/cuda-repo-9-0-local | grep .pub
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!sudo apt-get install cuda-9.0


--2021-09-08 10:07:48--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb [following]
--2021-09-08 10:07:49--  https://developer.nvidia.com/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?hEinC3zFyBeerPqbbacjOjwQ2zE5loqpySHYILhAGoO6hosuomSmVCnPPTILu0QR4R3lsATMTmt1dGhghk7zrfPzoQU-k_Yas

In [ ]:
%%shell
pwd

In [ ]:
%%shell
git clone https://github.com/Xtra-Computing/thundersvm.git
git submodule update --init src/test/googletest
# Build ThunderSVM

cd thundersvm
mkdir build
cd build
cmake ..
make -j
from importlib.machinery import SourceFileLoader
thundersvm = SourceFileLoader("thundersvm", "/content/thundersvm/python/thundersvm/thundersvm.py").load_module()

svc = svm.SVC(verbose=True)

In [ ]:
import os
owd = os.getcwd()
print(owd)

/content


In [ ]:

import pandas as pd
import thundersvm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import numpy as np

from keras.models import Sequential
from keras.layers import Dense

def read_data():
    df_test = pd.read_csv('2020-21//shifted_fpl.csv')
    df_train = pd.read_csv('2019-20//shifted_fpl.csv')
    shifted_feats = pd.read_csv('shifted.csv')['features']
    for df in df_train, df_test:
        for feat in shifted_feats:
            if feat in df.columns:
                df.drop(feat, axis=1, inplace=True)
    return df_test, df_train

def one_hot_encode(df_test):
    """[One hot encode all features but the players name and the kickoff time]

    Args:
        df ([type]): [description]

    Returns:
        [type]: [description]
    """
    cat = df_test.select_dtypes(exclude='number').columns.drop(['player_name', 'kickoff_time']) 
    df_test = pd.get_dummies(df_test, columns=cat, prefix=cat)
    return df_test




def plot_results(true_value, predicted_value, title):
    plt.figure(figsize=(10,10))
    plt.scatter(true_value, predicted_value, c='crimson')
    p1 = max(max(predicted_value), max(true_value))
    p2 = min(min(predicted_value), min(true_value))
    plt.plot([p1, p2], [p1, p2], 'b-')
    plt.xlabel('True Values', fontsize=15)
    plt.ylabel('Predictions', fontsize=15)
    plt.title(title)
    plt.axis('equal')
    plt.show()

    
def regression_results(y_true, y_pred, model_string):
    print('# <><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>')
    print(model_string)
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)
    print('explained_variance: ', round(explained_variance,4))    
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))

def scaled_metrics(regressor, X_train, y_train, X_test, y_test, scl_y, GW, model_str, epochs = None):
    if model_str is 'Scaled Net':
        regressor.fit(X_train, y_train, epochs=20)
    else:
        regressor.fit(X_train, y_train)
    y_train_rescaled = np.round(scl_y.inverse_transform(y_train))
    y_test_rescaled = np.round(scl_y.inverse_transform(y_test))
    y_pred_test = np.round(scl_y.inverse_transform(regressor.predict(X_test)))
    y_pred_train = np.round(scl_y.inverse_transform(regressor.predict(X_train)))
    regression_results(y_test_rescaled, y_pred_test, model_str)
    plot_results(y_test_rescaled, y_pred_test, title = f'Testing, 2020 Gameweek {GW}')
    regression_results(y_train_rescaled, y_pred_train, model_str)
    plot_results(y_train_rescaled, y_pred_train, title = f'Training, 2019 + 2020 GW: {GW - 1}')

def test_scaled_LR_model(X_train, y_train, X_test, y_test, scl_y, GW):
    """[This function tests a linear regression model on the provided data]

    Args:
        X_train ([type]): [The training predictors]
        X_test ([type]): [The testing predictors]
        y_train ([type]): [The testing predictors]
        y_test ([type]): [The training response]
    """
    
    regressor = LinearRegression(normalize=False, n_jobs=-1)
    scaled_metrics(regressor, X_train, y_train, X_test, y_test, scl_y, GW, 'Scaled LR')
    
def test_scaled_SVR_model(X_train, y_train, X_test, y_test, std_scaler_Y, GW):
    """[This function tests a linear regression model on the provided data]

    Args:
        X_train ([type]): [The training predictors]
        X_test ([type]): [The testing predictors]
        y_train ([type]): [The testing predictors]
        y_test ([type]): [The training response]
    """
    regressor = SVR(C=100, epsilon=0.01, gamma=0.005, verbose=True, kernel='rbf') # Note: This was fitted to scaled data
    scaled_metrics(regressor, X_train, y_train, X_test, y_test, std_scaler_Y, GW, 'Scaled SVR')
    

def plot_history(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean Square Error')
    plt.plot(hist['epoch'], hist['mse'], label='Train Error')
    plt.plot(hist['epoch'], hist['val_mse'], label = 'Val Error')
    plt.legend()
    plt.ylim([0,0.05])
    
    
def test_scaled_net(X_train, y_train, X_test, y_test, std_scale_Y_train, std_scale_Y_test, GW):
    """[This is very much just a]

    Args:
        X_train ([type]): [description]
        y_train ([type]): [description]
        X_test ([type]): [description]
        y_test ([type]): [description]
        std_scale_Y_train ([type]): [description]
        std_scale_Y_test ([type]): [description]
        GW ([type]): [description]
    """    
    regressor = Sequential()
    regressor.add(Dense(500, activation= "relu",))
    regressor.add(Dense(250, activation= "relu"))
    regressor.add(Dense(1))
    regressor.compile(loss= "mse" , optimizer="adam", metrics=["mse"])
    history = regressor.fit(X_train, y_train, epochs=20, validation_split=0.2)
    y_train_rescaled = np.round(std_scale_Y_train.inverse_transform(y_train))
    y_test_rescaled = np.round(std_scale_Y_test.inverse_transform(y_test))
    y_pred_test = np.round(std_scale_Y_test.inverse_transform(regressor.predict(X_test)))
    y_pred_train = np.round(std_scale_Y_train.inverse_transform(regressor.predict(X_train)))
    regression_results(y_test_rescaled, y_pred_test, 'Neural Net')
    plot_results(y_test_rescaled, y_pred_test, title = f'Testing, 2020 Gameweek {GW}')
    regression_results(y_train_rescaled, y_pred_train, 'Neural Net')
    plot_results(y_train_rescaled, y_pred_train, title = f'Training, 2019 + 2020 GW: {GW - 1}')
    plot_history(history)
 
    
# %%
GW = 3
df_test, df_train = read_data()
min_date = df_test[df_test['GW'] == GW]['kickoff_time'].min() # The first date of the gameweek
df_scl = df_train.append(df_test[df_test['GW'] <= GW]) # Only includes up to gameweek 
num_feats = df_scl.select_dtypes(include='number').drop(columns = ['total_points_shift', 'GW']).columns
std_scaler_X, std_scaler_Y = StandardScaler(), StandardScaler()
df_scl[num_feats] = std_scaler_X.fit_transform(df_scl[num_feats])
df_scl['total_points_shift'] = std_scaler_Y.fit_transform(df_scl['total_points_shift'].to_numpy().reshape(-1, 1))
df_scl = one_hot_encode(df_scl)
df_train, df_test = df_scl[df_scl['kickoff_time'] < min_date], df_scl[df_scl['kickoff_time'] >= min_date] 
X_train = df_train.drop(columns = ['player_name', 'GW', 'total_points_shift', 'kickoff_time']) 
y_train = df_train['total_points_shift']
X_test = df_test.drop(columns = ['player_name', 'GW', 'total_points_shift', 'kickoff_time'])
y_test = df_test['total_points_shift']
test_scaled_LR_model(X_train, y_train, X_test, y_test, std_scaler_Y, GW)
test_scaled_SVR_model(X_train, y_train, X_test, y_test, std_scaler_Y, GW)

SyntaxError: ignored